# Setup Environment and Download Data
## BƯỚC 1: Clone code và cài đặt package

In [18]:
import os
os.chdir('/content')
!pwd
!rm -rf /content/python_imageNet
!git clone -b vu https://github.com/CodeWithVu/python_imageNet.git /content/python_imageNet
%cd /content/python_imageNet
!pip install -q -e .


/content
Cloning into '/content/python_imageNet'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 64 (delta 16), reused 56 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 1.89 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/python_imageNet
  Preparing metadata (setup.py) ... done


## BƯỚC 2: Upload và cấu hình Kaggle API
### Hướng dẫn lấy kaggle.json:
1. Vào https://www.kaggle.com
2. Settings → API → "Create New API Token"
3. Tải file `kaggle.json` về máy
4. Chạy cell dưới đây và upload file kaggle.json khi được yêu cầu

In [19]:
import os, json
os.makedirs('/root/.kaggle', exist_ok=True)
creds = {"username":"nguynanhvdct122c4","key":"500061e5f39de8aed58366913a2ce3e9"}
with open('/root/.kaggle/kaggle.json','w') as f:
    json.dump(creds, f)
!chmod 600 /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json          # kiểm tra
!kaggle competitions list        

{"username": "nguynanhvdct122c4", "key": "500061e5f39de8aed58366913a2ce3e9"}ref                                                                                 deadline             category              reward  teamCount  userHasEntered  
----------------------------------------------------------------------------------  -------------------  ---------------  -----------  ---------  --------------  
https://www.kaggle.com/competitions/hull-tactical-market-prediction                 2025-12-15 23:59:00  Featured         100,000 Usd       2421           False  
https://www.kaggle.com/competitions/vesuvius-challenge-surface-detection            2026-02-13 23:59:00  Research         100,000 Usd        132           False  
https://www.kaggle.com/competitions/google-tunix-hackathon                          2026-01-12 23:59:00  Featured         100,000 Usd         51           False  
https://www.kaggle.com/competitions/csiro-biomass                                   2026-01-28 23:59:00  Rese

## BƯỚC 3: Tải dataset AID từ Kaggle (chạy trên cloud)

In [20]:
# Kiểm tra nếu data đã tải rồi thì skip
if not os.path.exists('./dataset/AID'):
    print(" Đang tải dataset từ Kaggle...")
    !kaggle datasets download -d jiayuanchengala/aid-scene-classification-datasets -p ./dataset --force
    
    print(" Đang giải nén...")
    !unzip -q ./dataset/aid-scene-classification-datasets.zip -d ./dataset
    
    print("Dataset ready!")
else:
    print(" Dataset already exists")

sdir = './dataset/AID'
print(f"📁 Data path: {sdir}")

 Đang tải dataset từ Kaggle...
Dataset URL: https://www.kaggle.com/datasets/jiayuanchengala/aid-scene-classification-datasets
License(s): CC0-1.0
 99% 2.43G/2.45G [00:59<00:00, 29.9MB/s]
100% 2.45G/2.45G [00:59<00:00, 43.9MB/s]
 Đang giải nén...
Dataset ready!
📁 Data path: ./dataset/AID


---
# Main Training Pipeline
## Import các module cần thiết

In [21]:
from doan.data_loading import create_dataframe
from doan.data_analysis import analyze_dataset
from doan.data_trimming import trim_dataframe
from doan.generators import create_generators
from doan.model_creation import create_model
from doan.callbacks_setup import create_callbacks  # quan trọng
from doan.training import train_model
from doan.visualization import plot_training_data, show_image_samples
from doan.evaluation import predictor, save_model


## Load và phân tích dataset

In [22]:
df = create_dataframe(sdir)
print(f'Dataset loaded: {df.shape}')

classes, class_count = analyze_dataset(df)
print(f'🏷️  Found {class_count} classes')
print(f'Classes: {classes[:5]}... (showing first 5)')

Dataset loaded: (10000, 2)
            CLASS               IMAGE COUNT 
           Airport                  360     
           BareLand                 310     
        BaseballField               220     
            Beach                   400     
            Bridge                  360     
            Center                  260     
            Church                  240     
          Commercial                350     
       DenseResidential             410     
            Desert                  300     
           Farmland                 370     
            Forest                  250     
          Industrial                390     
            Meadow                  280     
      MediumResidential             290     
           Mountain                 340     
             Park                   350     
           Parking                  390     
          Playground                370     
             Pond                   420     
             Port           

## Cân bằng dữ liệu (Data Trimming)

In [23]:
# Trim dataset để balance
max_samples = 500  # Số samples tối đa mỗi class
min_samples = 50   # Số samples tối thiểu mỗi class
df_trimmed = trim_dataframe(df, max_samples, min_samples, 'labels')
print(f'  Dataset after trimming: {df_trimmed.shape}')

after trimming, the maximum samples in any class is now  500  and the minimum samples in any class is  50
  Dataset after trimming: (10000, 2)


## Chia dataset và tạo generators

In [24]:
from sklearn.model_selection import train_test_split

# Split dataset
train_df, temp_df = train_test_split(df_trimmed, test_size=0.3, random_state=42, stratify=df_trimmed['labels'])
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['labels'])

print(f' Train: {len(train_df)}, Valid: {len(valid_df)}, Test: {len(test_df)}')

# Create generators
img_size = (224, 224)
batch_size = 32
train_gen, valid_gen, test_gen, classes, class_count, test_steps = create_generators(train_df, valid_df, test_df, img_size, batch_size)
print(f' Generators created')

 Train: 7000, Valid: 1500, Test: 1500
Found 7000 validated image filenames belonging to 30 classes.
Found 1500 validated image filenames belonging to 30 classes.
Found 1500 validated image filenames belonging to 30 classes.
test batch size:  75  test steps:  20  number of classes:  30
 Generators created


## Tạo mô hình EfficientNetB3

In [25]:
# Create EfficientNetB3 model
model = create_model(img_size + (3,), class_count)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │      1,080 │ stem_conv_pad[0]… │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        160 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        360 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        160 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 40)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 40)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 10)  │        410 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 40)  │        440 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 40)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        960 │ block1a_se_excit

 Total params: 11,190,861 (42.69 MB)

 Trainable params: 11,100,486 (42.34 MB)

 Non-trainable params: 90,375 (353.03 KB)

## Huấn luyện mô hình

In [26]:
# Setup callbacks
epochs = 20
ask_epoch = 10
callbacks = create_callbacks(model, epochs, ask_epoch)

# Train model
print("🚀 Starting training...")
history = train_model(model, train_gen, valid_gen, epochs, callbacks)
print("✅ Training completed!")

AttributeError: property 'model' of 'ASK' object has no setter

## Visualization

In [ ]:
# Plot training history
plot_training_data(history, 0)

# Show sample images
show_image_samples(train_gen)

## Đánh giá và lưu mô hình

In [ ]:
# Predict và evaluate
errors, tests = predictor(model, test_gen, test_steps, classes)

# Save model to Colab (hoặc Google Drive nếu đã mount)
save_dir = '/content/models'  # Hoặc '/content/drive/MyDrive/DoAn_Models' nếu mount Drive
os.makedirs(save_dir, exist_ok=True)

# Thay accuracy thực tế sau khi test
save_model(model, 'efficientnet_b3_aid', 95.5, save_dir)
print(f"✅ Model saved to {save_dir}")